In [1]:
import sys
import numpy as np
import pandas as pd

# import internal modules
sys.path.insert(1, '../src')
from utils.accuracy import MAE, MAPE, MSE, Error, AbsoluteError, PercentageError, AbsolutePercentageError
from execute.accuracy_evaluation import accuracy_table, accuracy_table_i

In [2]:
# read data
tse1 = pd.read_csv("../data/processed/tidy_tse1.csv", index_col=[0, 1, 2])
df = pd.read_csv("../data/processed/tidy_df.csv", index_col=[0, 1, 2])
y_test = pd.read_csv("../assets/y_hats/univariate/y_test.csv", index_col=[0, 1, 2])

# read y_hat csv
y_hats_all = pd.read_csv("./../assets/y_hats/y_hats_all.csv", index_col=[0, 1, 2])

# y_hat_umlp_tuned = pd.read_csv("./../assets/y_hats/univariate/y_hat_umlp-tuned_0905.csv", index_col=[0, 1, 2])
# y_hat_umlp_tuned.columns = ['y_hat_umlp_tuned']

y_hat_mmlp_tuned = pd.read_csv("./../assets/y_hats/multivariate/y_hat_mmlp_tuned_scaled.csv", index_col=[0, 1, 2])
y_hat_mmlp_tuned.columns = ['y_hat_mmlp_tuned_scaled']

In [4]:
y_hat_mmlp_tuned

y_hat_mmlp_tuned_scaled
企業名   会計年度 四半期                         
あらた   2018 Q1                 60.607697
           Q2                 75.818980
           Q3                 61.267445
           Q4                 66.711740
      2019 Q1                108.427100
...                                 ...
ｅＢＡＳＥ 2019 Q4                -37.442333
      2020 Q1                 -7.081633
           Q2                  4.746031
           Q3               -235.246170
           Q4                 34.971806

[13068 rows x 1 columns]

In [5]:
y_hats_all

y_test    y_hat_rw  y_hat_sarima_br  y_hat_sarima_f  \
企業名   会計年度 四半期                                                            
あらた   2018 Q1   105.420000   69.340000       122.565740      122.449521   
           Q2    90.440000  105.420000        79.897110       79.998967   
           Q3   106.240000   90.440000       119.386680      119.551232   
           Q4    99.220000  106.240000        60.222049       59.996949   
      2019 Q1   116.850000   99.220000       127.203963      125.133255   
...                    ...         ...              ...             ...   
ｅＢＡＳＥ 2019 Q4    18.657982    5.290000        51.406785       36.300485   
      2020 Q1     2.010000   18.657982       -68.232333       -0.020788   
           Q2     3.810000    2.010000        19.227708       11.637899   
           Q3     3.070000    3.810000      -317.707192        3.736002   
           Q4     9.989365    3.070000        34.835646       18.235485   

                y_hat_sarima_g  y_hat_umlp  y_hat_mlm1   y_hat_mlm2  \
企業名   会計年度 四半期                                                        
あらた   2018 Q1       130.595361   111.25781  142.213501   113.490222   
           Q2        94.544504    84.75139   92.463386   157.027451   
           Q3       119.287623   114.48496  118.419913    84.545219   
           Q4        65.424073    79.35591   72.589673   111.753597   
      2019 Q1       126.004465   113.64266   98.036624   126.181789   
...                        ...         ...         ...          ...   
ｅＢＡＳＥ 2019 Q4        88.758959  -138.77140  -82.273856  1361.294861   
      2020 Q1      -980.636406  -454.12380  161.435880   -96.581389   
           Q2      -201.877901 -1139.42900 -129.129435   638.576354   
           Q3      -249.404402  -299.76132  -55.248900    47.373257   
           Q4        72.472296  -176.82642    8.203400    55.521833   

                y_hat_mlm3  y_hat_mlm4  
企業名   会計年度 四半期                          
あらた   2018 Q1   117.339606  263.859095  
           Q2   147.870649  186.013135  
           Q3    84.910297  -90.797436  
           Q4    83.042883   43.663635  
      2019 Q1   135.395355  137.056577  
...                    ...         ...  
ｅＢＡＳＥ 2019 Q4   832.541667   31.609783  
      2020 Q1  -108.274805   -8.705827  
           Q2   562.144933  -12.323521  
           Q3    -3.808247   14.509884  
           Q4   130.403399    8.147153  

[13068 rows x 10 columns]

In [6]:
y_hats_all_tuned = pd.concat([y_hats_all.loc[idx], y_hat_mmlp_tuned], axis=1)
y_hats_all_tuned

y_test    y_hat_rw  y_hat_sarima_br  y_hat_sarima_f  \
企業名   会計年度 四半期                                                            
あらた   2018 Q1   105.420000   69.340000       122.565740      122.449521   
           Q2    90.440000  105.420000        79.897110       79.998967   
           Q3   106.240000   90.440000       119.386680      119.551232   
           Q4    99.220000  106.240000        60.222049       59.996949   
      2019 Q1   116.850000   99.220000       127.203963      125.133255   
...                    ...         ...              ...             ...   
ｅＢＡＳＥ 2019 Q4    18.657982    5.290000        51.406785       36.300485   
      2020 Q1     2.010000   18.657982       -68.232333       -0.020788   
           Q2     3.810000    2.010000        19.227708       11.637899   
           Q3     3.070000    3.810000      -317.707192        3.736002   
           Q4     9.989365    3.070000        34.835646       18.235485   

                y_hat_sarima_g  y_hat_umlp  y_hat_mlm1   y_hat_mlm2  \
企業名   会計年度 四半期                                                        
あらた   2018 Q1       130.595361   111.25781  142.213501   113.490222   
           Q2        94.544504    84.75139   92.463386   157.027451   
           Q3       119.287623   114.48496  118.419913    84.545219   
           Q4        65.424073    79.35591   72.589673   111.753597   
      2019 Q1       126.004465   113.64266   98.036624   126.181789   
...                        ...         ...         ...          ...   
ｅＢＡＳＥ 2019 Q4        88.758959  -138.77140  -82.273856  1361.294861   
      2020 Q1      -980.636406  -454.12380  161.435880   -96.581389   
           Q2      -201.877901 -1139.42900 -129.129435   638.576354   
           Q3      -249.404402  -299.76132  -55.248900    47.373257   
           Q4        72.472296  -176.82642    8.203400    55.521833   

                y_hat_mlm3  y_hat_mlm4  y_hat_mmlp_tuned_scaled  
企業名   会計年度 四半期                                                   
あらた   2018 Q1   117.339606  263.859095                60.607697  
           Q2   147.870649  186.013135                75.818980  
           Q3    84.910297  -90.797436                61.267445  
           Q4    83.042883   43.663635                66.711740  
      2019 Q1   135.395355  137.056577               108.427100  
...                    ...         ...                      ...  
ｅＢＡＳＥ 2019 Q4   832.541667   31.609783               -37.442333  
      2020 Q1  -108.274805   -8.705827                -7.081633  
           Q2   562.144933  -12.323521                 4.746031  
           Q3    -3.808247   14.509884              -235.246170  
           Q4   130.403399    8.147153                34.971806  

[13068 rows x 11 columns]

In [7]:
indicators = {"MAPE": MAPE}

In [8]:
a = accuracy_table(y_hats_all_tuned["y_test"], y_hats_all_tuned, indicators)
a

,y_test,y_hat_rw,y_hat_sarima_br,y_hat_sarima_f,y_hat_sarima_g,y_hat_umlp,y_hat_mlm1,y_hat_mlm2,y_hat_mlm3,y_hat_mlm4,y_hat_mmlp_tuned_scaled
MAPE,0.0,4.313953,9.708306,2.823022,6.123333,14.806769,5.778863,14.285043,15.209637,31.944189,3.837378


multivariate MLP tuned_scaleはいい感じ?

In [9]:
ai = accuracy_table_i(y_hats_all_tuned["y_test"], y_hats_all_tuned, indicators)
ai

,,y_test,y_hat_rw,y_hat_sarima_br,y_hat_sarima_f,y_hat_sarima_g,y_hat_umlp,y_hat_mlm1,y_hat_mlm2,y_hat_mlm3,y_hat_mlm4,y_hat_mmlp_tuned_scaled
あらた,MAPE,0.0,0.326915,0.209451,0.199988,0.202024,0.144319,0.200535,0.298360,0.273272,1.057403,0.280263
いすゞ自動車,MAPE,0.0,1.033565,0.674522,0.596406,0.769031,0.627432,1.406949,0.509453,0.529282,3.667137,0.838838
いなげや,MAPE,0.0,1.923584,0.933579,1.239337,1.253766,1.545034,2.690945,1.532201,1.594809,2.358227,2.173016
かどや製油,MAPE,0.0,0.973943,0.612928,0.585215,0.717560,0.756591,0.964528,0.372779,0.348536,1.999852,0.833886
きんでん,MAPE,0.0,1.548829,0.151986,0.132039,0.169481,0.308766,0.184258,0.283411,0.248188,0.438089,0.232925
...,...,...,...,...,...,...,...,...,...,...,...,...
ＶＴホールディングス,MAPE,0.0,1.026698,0.980742,0.952420,0.669884,1.394830,9.619477,1.909067,2.060582,10.090009,2.045024
ＷＤＢホールディングス,MAPE,0.0,0.204655,8.063783,0.242735,10.715042,4.547131,2.983076,3.794508,3.198188,1.255252,1.060657
ＷＯＷＯＷ,MAPE,0.0,1.720739,11.013297,0.509329,17.527644,14.660058,26.716439,110.643144,104.422490,10.399048,7.501756
ＺＯＺＯ,MAPE,0.0,0.595866,0.821336,0.431233,1.034765,2.980114,2.381338,3.660596,2.500938,2.073945,1.437206


In [10]:
ai.describe()

,y_test,y_hat_rw,y_hat_sarima_br,y_hat_sarima_f,y_hat_sarima_g,y_hat_umlp,y_hat_mlm1,y_hat_mlm2,y_hat_mlm3,y_hat_mlm4,y_hat_mmlp_tuned_scaled
count,1089.0,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000
mean,0.0,4.313953,9.708306,2.823022,6.123333,14.806769,5.778863,14.285043,15.209637,31.944189,3.837378
std,0.0,24.340679,225.215044,10.140076,88.594166,342.113625,50.936578,305.926501,317.366629,234.626426,17.536818
min,0.0,0.108712,0.049801,0.079664,0.071814,0.108270,0.051167,0.094949,0.090533,0.185546,0.107655
25%,0.0,0.633784,0.468086,0.496089,0.507989,0.586712,0.629979,0.663215,0.677769,1.664993,0.560870
50%,0.0,1.307412,0.978257,0.994169,1.056402,1.064089,1.286960,1.464974,1.552599,4.958196,1.144925
75%,0.0,2.604576,1.865236,1.934714,2.136072,2.290723,2.953894,3.314285,3.480510,13.595323,2.279961
max,0.0,492.518013,7428.137986,172.733304,2896.696746,11262.841325,1627.680038,10088.018752,10460.852271,5240.955894,348.793877


MMLP tuned scaleは、1を超えるエラーが低く抑えられてる?(Upper boundすると意味なさそう?)

ベスト予測の手法別企業カウント

In [17]:
for i in ai.columns[1:]:
    print(i)
    print((ai.iloc[:, 1:].idxmin(axis=1) == i).sum())

y_hat_rw
122
y_hat_sarima_br
223
y_hat_sarima_f
144
y_hat_sarima_g
128
y_hat_umlp
169
y_hat_mlm1
79
y_hat_mlm2
52
y_hat_mlm3
44
y_hat_mlm4
7
y_hat_mmlp_tuned_scaled
121


tuningによる各予測サンプルごとのエラーを見てみる

In [11]:
error_p_abs = []
for i in y_hats_all_tuned.columns:
    error_p_abs.append(AbsolutePercentageError(y_hats_all_tuned["y_test"], y_hats_all_tuned[i]))
error_p_abs = pd.DataFrame(error_p_abs).T
error_p_abs.columns = y_hats_all_tuned.columns
error_p_abs.describe()

,y_test,y_hat_rw,y_hat_sarima_br,y_hat_sarima_f,y_hat_sarima_g,y_hat_umlp,y_hat_mlm1,y_hat_mlm2,y_hat_mlm3,y_hat_mlm4,y_hat_mmlp_tuned_scaled
count,13068.0,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000
mean,0.0,4.313953,9.708306,2.823022,6.123333,14.806769,5.778863,14.285043,15.209637,31.944189,3.837378
std,0.0,83.569621,778.990262,33.688884,305.682629,1179.741370,171.827446,1057.593320,1096.885113,715.645658,56.772584
min,0.0,0.000000,0.000127,0.000035,0.000039,0.000044,0.000121,0.000355,0.000041,0.000266,0.000018
25%,0.0,0.240803,0.186371,0.177526,0.181412,0.265654,0.224952,0.234663,0.242207,0.641522,0.229096
50%,0.0,0.567493,0.457172,0.461149,0.464577,0.621202,0.574726,0.608518,0.639819,1.843934,0.519802
75%,0.0,1.313804,1.052149,1.085723,1.154855,1.274750,1.495443,1.622538,1.728114,6.113956,1.227357
max,0.0,5895.000000,88978.127205,2068.166620,34626.006762,134665.300000,19193.225310,120789.437098,125258.155053,50075.354129,3937.530700


ベスト予測の手法別予測サンプルカウント

In [12]:
for i in error_p_abs.columns[1:]:
    print(i)
    print((error_p_abs.iloc[:, 1:].idxmin(axis=1) == i).sum())

y_hat_rw
1690
y_hat_sarima_br
1237
y_hat_sarima_f
1547
y_hat_sarima_g
1400
y_hat_umlp
1498
y_hat_mlm1
1381
y_hat_mlm2
997
y_hat_mlm3
1032
y_hat_mlm4
811
y_hat_mmlp_tuned_scaled
1475


10%くらいはMMLPが一番いいけど、、数はSARIMA F、Random walk, UMLPには負けてる

Upper boundを適用してみる

In [18]:
#Upper bound
error_p_abs_ub = error_p_abs.copy()
error_p_abs_ub[error_p_abs_ub > 1] = 1

In [19]:
error_p_abs_ub.describe()

,y_test,y_hat_rw,y_hat_sarima_br,y_hat_sarima_f,y_hat_sarima_g,y_hat_umlp,y_hat_mlm1,y_hat_mlm2,y_hat_mlm3,y_hat_mlm4,y_hat_mmlp_tuned_scaled
count,13068.0,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000,13068.000000
mean,0.0,0.584368,0.526985,0.526749,0.534064,0.603029,0.585543,0.597033,0.609444,0.805381,0.566839
std,0.0,0.362200,0.364672,0.367901,0.371721,0.354113,0.371048,0.371075,0.370876,0.313807,0.362498
min,0.0,0.000000,0.000127,0.000035,0.000039,0.000044,0.000121,0.000355,0.000041,0.000266,0.000018
25%,0.0,0.240803,0.186371,0.177526,0.181412,0.265654,0.224952,0.234663,0.242207,0.641522,0.229096
50%,0.0,0.567493,0.457172,0.461149,0.464577,0.621202,0.574726,0.608518,0.639819,1.000000,0.519802
75%,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
# large error count
(error_p_abs_ub == 1).sum()

y_test                        0
y_hat_rw                   4209
y_hat_sarima_br            3469
y_hat_sarima_f             3565
y_hat_sarima_g             3743
y_hat_umlp                 4122
y_hat_mlm1                 4490
y_hat_mlm2                 4692
y_hat_mlm3                 4883
y_hat_mlm4                 8468
y_hat_mmlp_tuned_scaled    3955
dtype: int64

In [21]:
# large error propotion
(error_p_abs_ub == 1).sum() / len(error_p_abs_ub)

y_test                     0.000000
y_hat_rw                   0.322084
y_hat_sarima_br            0.265458
y_hat_sarima_f             0.272804
y_hat_sarima_g             0.286425
y_hat_umlp                 0.315427
y_hat_mlm1                 0.343587
y_hat_mlm2                 0.359045
y_hat_mlm3                 0.373661
y_hat_mlm4                 0.647995
y_hat_mmlp_tuned_scaled    0.302648
dtype: float64

UMLPよりはすくないけど、、Univariate Linearよりは多い。

ヘタクソなやつを特定

In [23]:
error_p_abs["y_hat_mmlp_tuned_scaled"].sort_values(ascending=False)[:50]

企業名              会計年度  四半期
山田コンサルティンググループ   2019  Q1     3937.530700
テイクアンドギヴ・ニーズ     2020  Q3     3025.496444
オカムラ             2020  Q1     2610.399300
タムラ製作所           2019  Q4     1726.533700
大阪製鉄             2020  Q3     1209.751260
第一三共             2020  Q4     1043.950875
東京機械製作所          2018  Q2     1023.730615
タムラ製作所           2019  Q1      634.419600
ＴＰＲ              2019  Q4      618.390091
さくらインターネット       2020  Q1      593.111196
日立建機             2020  Q2      478.491250
タカミヤ             2020  Q4      467.686250
東海旅客鉄道           2020  Q3      461.741043
サニックス            2018  Q3      431.947000
旭化成              2019  Q4      386.271171
はせがわ             2020  Q4      384.707700
さくらインターネット       2019  Q3      380.201052
                 2020  Q2      363.357621
ソースネクスト          2020  Q4      312.604762
東祥               2020  Q4      298.487672
三機工業             2020  Q1      267.916524
ホッカンホールディングス     2020  Q1      258.326256
日水製薬             2019  Q1      255.334967
日本水産   

時間かかりすぎて多っぽいからtrialを300-->100に減らしたけど、時間がかかってたのはepoch10000が原因かも?
trial 300, epoch 1000ならまだ改善かつ時間をかけずに行けるかも??